<a href="https://colab.research.google.com/github/xuxiufeng/Deep_Learning_with_Python/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gan generator network

In [73]:
import keras
from keras import layers
import numpy as np

In [74]:
latent_dim = 32
height = 32
width = 32
channels = 3
generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_5 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_1 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      1048

## The GAN discriminator network

In [75]:
import tensorflow as tf

In [76]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = tf.keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_26 (LeakyReLU)  (None, 30, 30, 128)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_26 (Conv2D)          (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 6, 6, 128)         0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Adversarial network

In [77]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,)) 
gan_output = discriminator(generator(gan_input)) 
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = tf.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8) 
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## Download the data

In [78]:
import os
from keras.preprocessing import image


In [79]:
(x_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape((x_train.shape[0],) +(height, width, channels)).astype('float32') / 255
iterations = 10 # 10000
batch_size = 20
save_dir = '/content/'
start = 0

In [80]:
for step in range(iterations):
    print('Progress: {}%'.format(round((step+1)/iterations*100)))
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size 
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    start += batch_size
    if start > len(x_train) - batch_size:
       start = 0
    if step % 100 == 0:
       gan.save_weights('gan.h5')
       print('discriminator loss:', d_loss)
       print('adversarial loss:', a_loss)
       img = image.array_to_img(generated_images[0] * 255., scale=False)
       img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))
       img = image.array_to_img(real_images[0] * 255., scale=False)
       img.save(os.path.join(save_dir,'real_frog' + str(step) + '.png'))

Progress: 10%
discriminator loss: 0.7019752264022827
adversarial loss: 0.653041422367096
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%


## Reference
* Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009
* Chollet, F., (2017), Deep Learning, ISBN: 9781638352044, 1638352046